# SATURDAY - BUDGET CONSTRANED

### SETUP

In [1]:
import time
beginning_time = time.time()

import sys
sys.path.append("../Code/")
from utils import performance

import os
DATA_DIR = os.path.join('..', 'Data')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

start = time.time()

def timtime():
    print(">",int((time.time()- start)//60), "minutes elapsed")

#----------------------------------------------------#    
    
train_X = pd.concat([pd.read_pickle(os.path.join(DATA_DIR, 'train_X_1')), 
                     pd.read_pickle(os.path.join(DATA_DIR, 'train_X_2'))])
train_y = pd.read_pickle(os.path.join(DATA_DIR, 'train_y'))
valid_X = pd.read_pickle(os.path.join(DATA_DIR, 'valid_X'))
valid_y = pd.read_pickle(os.path.join(DATA_DIR, 'valid_y'))

test_X = pd.read_pickle(os.path.join(DATA_DIR, 'test_X'))

### LOAD MODELS

_Using Logistic Regression as CTR predictor_

In [2]:
import pickle

GMM_filename = "../Models/SatGMM4"
with open(GMM_filename, 'rb') as file:
    GMM = pickle.load(file)
    

dist_pred_filename = "../Models/SatXGDistPred2"
with open(dist_pred_filename, 'rb') as file:
    dp = pickle.load(file)
    

ctr_pred_filename = "../Models/clf3.pkl"
with open(ctr_pred_filename, 'rb') as file:
    ctr = pickle.load(file)
    
ctr_pred_filename = "../Models/tim_xgb_click.pkl"
with open(ctr_pred_filename, 'rb') as file:
    ctr2 = pickle.load(file)    

### Investigate the market price distributions

In [3]:
from scipy.stats import norm

In [4]:
d1 = {"mu":GMM.means_[0][0], "sig":GMM.covariances_[0][0][0]}
d2 = {"mu":GMM.means_[1][0], "sig":GMM.covariances_[1][0][0]}
d3 = {"mu":GMM.means_[2][0], "sig":GMM.covariances_[2][0][0]}
d4 = {"mu":GMM.means_[3][0], "sig":GMM.covariances_[3][0][0]}

In [9]:
from scipy.sparse import csr_matrix
train_X = csr_matrix(train_X) 
valid_X = csr_matrix(valid_X)
valid_distributions = dp.predict(valid_X)

# REPLICATE THE LINEAR BIDDING STRATEGY

In [12]:
from utils import new_performance

In [10]:
valid_ctr_2 = ctr2.predict(valid_X)

In [16]:
bids = 75275*valid_ctr_2

In [17]:
_ = new_performance(bids,valid_y)

               CTR: (0.1063)%
        num_clicks: 163
             spend: 6154819 (98.48)%
              aCPM: 20.2511112939
              aCPC: 37.7596257669
num_in_budget_wins: 153353
 ads_within_budget: 303925


# ADD SOME BUDGET CONSTRAINTS

In [14]:
budget = 6500*1000

In [25]:
N = valid_X.shape[0]
valid_means = np.zeros(N)
for i in range(N):
    valid_means[i] = GMM.means_[valid_distributions[i],0]
    
valid_means = np.expm1(valid_means)

In [30]:
valid_means[0:10]

array([ 23.52274264,  23.52274264,  66.9654455 ,  66.9654455 ,
         7.23808926,  23.52274264,  66.9654455 ,  23.52274264,
        66.9654455 ,  66.9654455 ])

In [27]:
np.sum(bids>valid_means)/N

0.51198486468701165

In [38]:
### this is the expected budget we think would be spent on the whole validation set
np.sum(valid_means[bids>valid_means])

print((100*np.sum(valid_means[bids>valid_means]))//budget,"%")

119.0 %


In [41]:
1/1.19

0.8403361344537815

In [39]:
### thus we want to reduce the bidding over time so that the budget is expected to last through the whole phase

In [86]:
TAPER = np.linspace(0.9,1.1, N)

constrained_bids = TAPER*bids

_ = new_performance(constrained_bids,valid_y)

               CTR: (0.1069)%
        num_clicks: 164
             spend: 6163621 (98.62)%
              aCPM: 20.2800723863
              aCPC: 37.583054878
num_in_budget_wins: 153459
 ads_within_budget: 303925


In [116]:
TAPER = np.linspace(0.79,1.04, N)

constrained_bids = TAPER*bids

_ = new_performance(constrained_bids,valid_y)

               CTR: (0.1148)%
        num_clicks: 164
             spend: 5467509 (87.48)%
              aCPM: 17.9896652135
              aCPC: 33.3384695122
num_in_budget_wins: 142872
 ads_within_budget: 303925
